In [16]:
import argparse
import pandas as pd
import dgl
import torch
import torch.nn.functional as F
import collections
from scipy.sparse import csr_matrix, vstack, save_npz
from sklearn.decomposition import PCA
from pathlib import Path
import numpy as np
from pprint import pprint
import json
from collections import Counter

# Hyperparameters

In [56]:
device = torch.device('cpu')

# Get id2gene

In [34]:
data_gene = pd.read_csv("data/mouse_Muscle1102_data.gz", compression='gzip', header=0,index_col=0)

In [35]:
id2gene = data_gene.index.values.tolist()
id2gene.sort()

# Get id2label

In [38]:
data_cell = pd.read_csv("data/mouse_Muscle1102_celltype.csv", dtype=np.str, header=0,index_col=0)

/tmp/ipykernel_2758419/1672907279.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data_cell = pd.read_csv("data/mouse_Muscle1102_celltype.csv", dtype=np.str, header=0,index_col=0)


In [46]:
cell = set()
cell_type_list = []

In [47]:
data_cell['Cell_type'] = data_cell['Cell_type'].map(str.strip)
cell_types = set(data_cell.values[:, 1])
cell_type_list.extend(data_cell.values[:, 1].tolist())

In [49]:
id2label = list(cell_types)
label_statistics = dict(collections.Counter(cell_type_list))

In [52]:
total_cell = sum(label_statistics.values())

# Filter Cell

In [53]:
for label, num in label_statistics.items():
    if num / total_cell <= 0.005:
        id2label.remove(label)  # remove exclusive labels

In [54]:
gene2id = {gene: idx for idx, gene in enumerate(id2gene)}
num_genes = len(id2gene)
# prepare unified labels
num_labels = len(id2label)
label2id = {label: idx for idx, label in enumerate(id2label)}

# Create Graph

In [57]:
graph = dgl.DGLGraph()

gene_ids = torch.arange(num_genes, dtype=torch.int32, device=device).unsqueeze(-1)
graph.add_nodes(num_genes, {'id': gene_ids})

In [58]:
graph

Graph(num_nodes=16422, num_edges=0,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={})

In [59]:
all_labels = []
matrices = []
num_cells = 0

In [62]:
cell2type = pd.read_csv("data/mouse_Muscle1102_celltype.csv",index_col=0)
cell2type.columns = ['cell', 'type']
cell2type['type'] = cell2type['type'].map(str.strip)
cell2type['id'] = cell2type['type'].map(label2id)

In [63]:
filter_cell = np.where(pd.isnull(cell2type['id']) == False)[0]
cell2type = cell2type.iloc[filter_cell]

In [65]:
all_labels += cell2type['id'].tolist()

## Cell&Gene Matrix

In [68]:
df = pd.read_csv("data/mouse_Muscle1102_data.gz", compression='gzip', index_col=0)
df = df.transpose(copy=True) 
df

,0610005C13Rik,0610009B22Rik,0610009E02Rik,0610009L18Rik,0610010F05Rik,0610010K14Rik,0610012G03Rik,0610030E20Rik,0610038B21Rik,0610039K10Rik,...,Galnt17,Bud23,Mettl27,Ccn4,Ccn5,Get1,Tut4,Rtl3,Tut7,Zup1
C_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.692061,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_1098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_1099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_1100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_1101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
df = df.iloc[filter_cell]
df = df.rename(columns=gene2id)
col = [c for c in df.columns if c in gene2id.values()]
df = df[col]

In [71]:
arr = df.to_numpy()
row_idx, col_idx = np.nonzero(arr > 0)  # intra-dataset index
non_zeros = arr[(row_idx, col_idx)]  # non-zero values
cell_idx = row_idx + graph.number_of_nodes()  # cell_index
gene_idx = df.columns[col_idx].astype(int).tolist()  # gene_index
info_shape = (len(df), num_genes)
info = csr_matrix((non_zeros, (row_idx, gene_idx)), shape=info_shape)
matrices.append(info)

num_cells += len(df)

In [72]:
ids = torch.tensor([-1] * len(df), dtype=torch.int32, device=device).unsqueeze(-1)
graph.add_nodes(len(df), {'id': ids})
graph.add_edges(cell_idx, gene_idx,
                {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})
graph.add_edges(gene_idx, cell_idx,
                {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})

In [73]:
graph

Graph(num_nodes=17524, num_edges=1169536,
      ndata_schemes={'id': Scheme(shape=(1,), dtype=torch.int32)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float32)})

## Cell&Gene Features

In [75]:
sparse_feat = vstack(matrices).toarray()  # cell-wise  (cell, gene)

In [79]:
sparse_feat.shape

(1102, 16422)

In [80]:
gene_pca = PCA(400, random_state=10086).fit(sparse_feat.T)
gene_feat = gene_pca.transform(sparse_feat.T)
gene_evr = sum(gene_pca.explained_variance_ratio_) * 100

In [81]:
sparse_feat = sparse_feat / (np.sum(sparse_feat, axis=1, keepdims=True) + 1e-6)
# use weighted gene_feat as cell_feat
cell_feat = sparse_feat.dot(gene_feat)
gene_feat = torch.from_numpy(gene_feat)  # use shared storage
cell_feat = torch.from_numpy(cell_feat)

graph.ndata['features'] = torch.cat([gene_feat, cell_feat], dim=0).type(torch.float).to(device)
labels = torch.tensor([-1] * num_genes + all_labels, dtype=torch.long, device=device)  # [gene_num+train_num]

# Random Permutation Cell index

In [83]:
per = np.random.permutation(range(num_genes, num_genes + num_cells))
test_ids = torch.tensor(per[:int(num_cells // ((1 - 0.2) / 0.2 + 1))]).to(device)
train_ids = torch.tensor(per[int(num_cells // ((1 - 0.2) / 0.2 + 1)):]).to(device)

# Normalize Weight

In [87]:
in_degrees = graph.in_degrees()
for i in range(graph.number_of_nodes()):
    src, dst, in_edge_id = graph.in_edges(i, form='all')
    if src.shape[0] == 0:
        continue
    edge_w = graph.edata['weight'][in_edge_id]
    graph.edata['weight'][in_edge_id] = in_degrees[i] * edge_w / torch.sum(edge_w)

In [91]:
# add self-loop
graph.add_edges(graph.nodes(), graph.nodes(),
                {'weight': torch.ones(graph.number_of_nodes(), dtype=torch.float, device=device).unsqueeze(1)})

In [92]:
graph.readonly()

/home/chenhuaguan/.local/lib/python3.9/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.readonly is deprecated in v0.5.
DGLGraph now always supports mutable operations like add_nodes and add_edges.
  return warnings.warn(message, category=category, stacklevel=1)
